In [58]:
import os 
import pandas as pd
import numpy as np

from LoadData import LoadData
from Filter import Filter
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

In [2]:
gap_animals = ["S7063", "S7064", "S7069"]
wt_animals = ["S7068", "S7070", "S7071"] 
gap_rec = [f for f in os.listdir() if "GAP.npy" in f]
wt_rec = [f for f in os.listdir() if "WT.npy" in f] 

In [3]:
# Here choose the animal recording and start/end
directory = '/Volumes/Macintosh HD/Users/gokceuzun/Desktop/4. SENE/Honors Project/'
filename = 'S7063_GAP.npy'
start = 15324481
end = 36959040

# Load the recording, slice and filter it
data = LoadData(directory, filename)
unfiltered_data = data.get_dat()[:, start:end + 1]
fltr = Filter(unfiltered_data)
filtered_data = np.array(fltr.butter_bandpass())

In [4]:
print("There are " + str(len(filtered_data[0])) + " many samples.") # Getting the correct number of samples

There are 21634560 many samples.


In [64]:
channels = ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16']

In [65]:
df = pd.DataFrame(filtered_data).T
df.set_axis(channels, axis="columns", inplace=True)

In [6]:
print("There are " + str(int(len(df)/1252)) + " many epochs.")

There are 17280 many epochs.


In [7]:
# Add epochs and the sample indices to the recording dataframe
epochs = []
for i in range(17280): # int(len(df)/1252)
    for j in range(1252): # no of samples per epoch
        e = "epoch" + str(i + 1)
        epochs.append(e)
df['epochs'] = epochs
df

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,14.517459,6.050171,9.636889,16.864971,11.159300,8.129794,0.847358,8.844793,13.181217,9.431838,12.489662,12.430479,13.903798,16.700808,5.349366,12.934711,epoch1
1,9.682026,-2.020716,3.714286,9.115627,6.424321,3.375708,0.007191,4.037892,12.174484,6.720867,7.484081,3.319452,6.870806,12.457773,-1.733461,9.000016,epoch1
2,9.376320,-13.766812,-1.509305,5.616394,4.717928,3.787080,0.637890,2.402899,10.294084,3.869864,-2.529693,-10.401293,-7.039740,2.940996,-1.975282,3.784293,epoch1
3,8.812900,-9.693198,-1.186084,8.444033,5.687863,2.525990,-1.989646,3.213470,9.267854,4.136354,-1.352342,-9.558710,-8.962019,2.358410,-3.258398,4.109588,epoch1
4,11.488102,-2.206936,4.440248,17.630630,10.679531,2.937450,-3.099315,8.274583,10.225103,8.609006,10.262874,1.316788,4.709829,7.621674,1.872643,12.689111,epoch1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21634555,-3.646540,-1.778447,-9.393458,-13.525131,-8.299972,1.362023,-0.264822,-7.636986,-2.278264,-2.539186,-17.708328,-28.356850,-21.146596,-11.046030,2188.363504,-3.566296,epoch17280
21634556,-4.962000,-1.549264,-10.945738,-13.697753,-8.539701,1.258169,-1.482945,-9.502496,-1.375945,-2.975565,-21.092770,-31.387665,-24.783426,-12.141887,2262.593972,-5.313137,epoch17280
21634557,-0.042612,-7.248449,-13.445747,-14.554149,-11.216752,4.843350,-0.983400,-6.757396,-3.477490,-1.782986,-17.033068,-26.788262,-18.592876,-12.903829,2194.342554,0.435350,epoch17280
21634558,0.317769,-14.125230,-7.831569,-12.576911,-8.109595,4.569688,0.346244,-2.951969,-9.401787,-5.021864,-12.397588,-18.879573,-12.258143,-9.157854,1595.389344,0.704171,epoch17280


#### An important step will be loading the entire recording and finding a way to store the results of each epoch so that you can easily extract features for particular time indices down the line for clustering and also keeping track of the indices that have been discarded for filtering so that the time indices don't get mixed up. 

In [8]:
# Keeping track of the indices that have been discarded for filtering so that the time indices don't get mixed up!
discarded_indices = fltr.noise_index
discarded_ch1 = discarded_indices[0]
ch = 'ch1'

n = ["no"]*len(df)
df_ch1 = df[[ch, 'epochs']]
df_ch1.insert(2, 'discarded', n)

for i in discarded_ch1:
    df_ch1.at[i, 'discarded'] = "yes"

df_ch1

,ch1,epochs,discarded
0,14.517459,epoch1,no
1,9.682026,epoch1,no
2,9.376320,epoch1,no
3,8.812900,epoch1,no
4,11.488102,epoch1,no
...,...,...,...
21634555,-3.646540,epoch17280,no
21634556,-4.962000,epoch17280,no
21634557,-0.042612,epoch17280,no
21634558,0.317769,epoch17280,no


In [21]:
# Which channels or epochs to use can be selected here: 
d = []
channels = ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16']

epochs = ['epoch1', 'epoch2', 'epoch3', 'epoch4', 'epoch5']

for epoch in epochs: # df.epochs
    e = df[df.epochs == epoch]
    #print(epoch)
    a = []
    for channel in channels:
        arr = e[channel].to_numpy()
        a.append(arr)
    d.append(a)
    
df_d = pd.DataFrame(d, columns=channels) 
df_d['epochs'] = epochs
df_d

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,"[14.517459411141314, 9.68202610913161, 9.37631...","[6.050170734109054, -2.0207160460076317, -13.7...","[9.63688947589803, 3.7142859877897805, -1.5093...","[16.864971058053964, 9.115626692360443, 5.6163...","[11.159300021033165, 6.424321460539171, 4.7179...","[8.129794494053032, 3.375708117759906, 3.78707...","[0.8473580547179342, 0.007191232725786811, 0.6...","[8.844793067805156, 4.037892191399959, 2.40289...","[13.181216879084008, 12.174484120039835, 10.29...","[9.431838352368654, 6.720866792806385, 3.86986...","[12.489661728481714, 7.484080742398889, -2.529...","[12.43047904931879, 3.319452440512472, -10.401...","[13.903798144624062, 6.87080645860974, -7.0397...","[16.700808484146297, 12.457772590411349, 2.940...","[5.349365644732483, -1.7334609100843918, -1.97...","[12.934711059559888, 9.00001607626519, 3.78429...",epoch1
1,"[36.4266790802226, 37.45161835525188, 36.29982...","[17.86926813303952, 10.832831376219836, 7.7981...","[17.373062464403958, 11.208706021879825, 4.759...","[19.368857307744587, 17.61401630303911, 13.627...","[15.657765799993287, 11.904772083306987, 7.934...","[3.931212761200225, 3.169270765225269, 0.96338...","[5.457504634287783, 3.3754144975655223, 0.9960...","[10.199214219449523, 8.591305421801986, 6.7654...","[19.809464391916222, 16.120034123821, 7.988776...","[12.973657905350242, 10.321479527024419, 1.962...","[12.423308377212972, 8.810642081665145, 2.3829...","[16.634932259569403, 10.080922395636092, 2.671...","[5.19427298882477, 4.201516153224029, 0.143936...","[-75.77322003201037, -81.75302162352028, -87.0...","[15.8066342418478, 8.606767405628048, 5.298414...","[6.845480844268209, 6.632760674044208, 1.92068...",epoch2
2,"[12.18718848007577, 12.418511515058585, 11.371...","[-2.9705696272221243, 7.212761223958914, 12.57...","[-8.482443964821652, -4.970348225200457, -2.76...","[-13.628134370476555, -9.327326853881704, -13....","[-0.3623393166640225, 3.017262757508239, 0.454...","[-6.054367122816045, -4.986288905589369, -5.33...","[3.971702121916909, 6.397293644777816, 7.82024...","[-10.992613392955578, -7.5363479576844075, -10...","[13.210989800391015, 17.61315812325442, 24.831...","[-13.447165090579508, -7.643289373531568, -0.6...","[16.19516514939972, 22.80997479624355, 28.9087...","[-5.899296868967301, 2.286931115255762, 7.7584...","[64.62876347871693, 71.1897023801094, 74.17420...","[47.75024909334192, 54.07229956698956, 59.3259...","[-2.355897118388694, 4.408866555097269, 17.776...","[-8.08796715456345, -6.917294175084411, -3.179...",epoch3
3,"[22.45868885159402, 26.375392629176368, 23.832...","[35.19874748429637, 25.15688647414171, 21.2061...","[16.806713043060682, 21.65107429174415, 17.286...","[11.52258381572238, 18.604994211082463, 16.006...","[18.711160134697217, 23.845868189236413, 21.91...","[11.74691086176005, 12.121683137451827, 10.093...","[20.47354979508988, 19.287697222610188, 16.298...","[20.14990211100938, 27.077086601137495, 25.325...","[-5.15292633496397, -1.1498515544579648, -4.26...","[4.014244471835262, 9.808018459871455, 5.50922...","[23.33501353142814, 31.862472654904675, 24.945...","[22.67005213796568, 27.20525553052873, 19.0430...","[12.19538418760882, 20.345319788688478, 12.351...","[32.11537582254555, 42.51703438483007, 36.8274...","[36.35912084543649, 28.34218709352538, 23.0620...","[6.373559548238479, 12.26601981476471, 8.92572...",epoch4
4,"[45.453638891809234, 43.81670438892779, 32.327...","[33.72922942124266, 33.22467792137823, 37.2872...","[80.40987155576158, 78.65233757551383, 70.2879...","[82.18952858916796, 83.19087822451888, 74.0666...","[76.67738813666313, 78.0648691848876, 68.55683...","[27.88888706462962, 25.8918433085489, 18.79166...","[22.830998780588935, 22.897828735725085, 18.84...","[92.30416034657131, 93.45628067489523, 84.8207...","[-9.329944815818173, -10.186481756488067, -12....","[26.087376845997646, 24.061222667330313, 22.15...","[52.29225996318509, 50.75793898648

#### The time domain features will be calculated for each epoch for each channel (and the channels determined by the correlation matrices).  
Time domain features: mean value, median, standard deviation, root mean square (RMS), kurtosis, skewness, variance, max, min etc.

In [22]:
type(df_d['ch1'][0])

numpy.ndarray

#### MEAN

In [51]:
df_mean = df_d[channels].applymap(lambda x: np.mean(x)).assign(epochs=epochs)
df_mean

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,-3.770756,-2.131234,-0.626729,-0.309846,-0.862681,-0.738643,-0.959474,1.171167,1.960721,-0.205772,-1.239892,-1.001668,-2.111918,-2.791813,-1.715558,-0.290409,epoch1
1,-0.557516,1.612477,0.461850,0.744766,0.696614,1.289945,0.529826,-1.488810,-1.462546,0.244264,-3.001878,0.708716,-1.675778,-5.274797,1.770704,0.284045,epoch2
2,2.979779,-0.738980,3.166900,-0.839767,0.219501,-0.765029,-0.131480,-1.437829,1.172585,0.182102,4.524732,-0.327206,2.896896,2.019089,-0.779044,-0.079738,epoch3
3,1.330424,-0.090113,-2.098208,0.581069,0.481700,0.494696,0.180204,4.129821,-1.334071,-0.257431,0.579844,0.817875,0.520933,5.279732,-0.206891,1.007959,epoch4
4,-3.998681,-0.581225,-1.006214,0.250752,-0.337519,-0.129592,-0.331523,-4.633996,-1.339969,-0.138560,-1.024686,-0.728711,-6.547684,0.762375,-0.437455,0.066635,epoch5


#### MEDIAN

In [52]:
df_median = df_d[channels].applymap(lambda x: np.median(x)).assign(epochs=epochs)
df_median

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,-1.035479,-1.458444,0.842895,-0.860899,0.157026,-0.319781,-0.039491,4.694507,2.158417,-0.501258,-2.056438,-1.627238,-4.107996,-2.854526,-2.248282,-0.447531,epoch1
1,-8.530433,1.969019,0.816216,0.857761,0.728788,0.350951,1.448204,-1.221537,-4.928229,0.242041,-3.108096,-0.470971,-2.733831,-28.542423,1.791518,-0.912300,epoch2
2,1.129845,0.234550,2.837518,-0.966532,1.090352,-0.676317,0.009699,-2.108000,0.261133,-0.315105,6.121446,-0.622597,-3.215256,-1.208495,-1.068444,-1.431813,epoch3
3,2.178920,0.388765,-3.744435,2.037423,1.600000,1.075494,0.432029,6.622035,-9.382379,-1.017640,0.927748,-0.845880,0.645242,9.152829,-1.461920,0.752632,epoch4
4,-4.007304,0.353610,1.155472,1.991171,2.077051,-0.018037,-0.417990,-2.681090,-5.122894,-0.083818,-1.661539,-1.076656,-6.752494,-5.470234,-1.249789,-0.567317,epoch5


#### VARIANCE

In [53]:
df_variance = df_d[channels].applymap(lambda x: np.var(x)).assign(epochs=epochs)
df_variance

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,383.205643,180.362917,199.607370,208.123032,189.718944,87.208493,64.834580,380.393569,244.112869,103.921904,162.435890,184.801174,289.126863,1111.796769,109.889965,121.515639,epoch1
1,593.945520,157.217608,190.799386,237.918524,189.430603,91.494735,56.362497,210.838943,173.890043,98.005175,274.963062,204.620499,339.685938,6962.666331,188.600136,124.946003,epoch2
2,389.880583,196.050454,534.021884,281.081855,212.671038,78.286402,54.976913,258.437687,175.853478,137.908277,998.328438,274.003808,1044.301248,1357.458520,191.555669,165.451822,epoch3
3,517.870765,373.606389,917.492780,1097.492443,905.070681,265.621807,108.749078,1272.846292,784.881461,347.977149,653.598047,886.139416,1031.534301,1367.258937,353.498584,644.810396,epoch4
4,542.619529,276.096486,603.144264,655.658350,603.111486,187.452635,84.717828,633.381474,339.271161,210.418694,483.809705,573.024783,1585.541129,2524.744740,252.143405,397.522170,epoch5


#### SKEWNESS

In [56]:
df_skewness = df_d[channels].applymap(lambda x: skew(x)).assign(epochs=epochs)
df_skewness

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,-0.673439,-0.144246,-0.186584,-0.000050,-0.274121,-0.236102,-0.338448,-0.635668,-0.126573,0.084779,0.107540,0.112818,0.335549,-0.289066,0.135518,0.283061,epoch1
1,0.414475,-0.052375,0.032116,0.379353,0.181528,0.376721,-0.177885,0.116938,0.643048,0.178962,0.037149,0.180272,0.748829,0.364382,0.042950,0.644702,epoch2
2,0.293563,-0.478686,-0.184869,0.270349,-0.251140,0.101190,-0.029117,0.389811,0.134506,0.162544,0.126772,0.138834,0.886890,0.144770,-0.260554,0.746939,epoch3
3,-0.097353,-0.373161,0.232217,-0.087788,-0.022154,-0.082055,-0.478676,-0.380218,0.657291,0.446577,-0.020903,0.115200,-0.084764,-0.673536,-0.133882,0.085410,epoch4
4,0.054804,-0.564033,-0.245091,-0.349322,-0.356556,-0.191608,-0.116264,-0.204996,0.322092,-0.325540,-0.153076,-0.236965,0.191591,0.198567,-0.272177,-0.039145,epoch5


#### KURTOSIS

In [59]:
df_kurtosis = df_d[channels].applymap(lambda x: kurtosis(x)).assign(epochs=epochs)
df_kurtosis

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,0.369396,-0.563674,-0.472220,-0.213044,-0.135801,0.059711,-0.379341,0.320152,-0.611987,-0.192781,-0.340003,-0.138784,-0.132678,-0.136679,-0.328658,0.317542,epoch1
1,-1.072892,0.094715,0.244594,0.799888,0.620956,0.241697,-0.711155,0.227734,-0.503600,0.461047,-0.392817,-0.177790,1.964304,-1.256976,1.785346,1.011352,epoch2
2,-0.450707,1.573958,0.502773,0.622065,0.360449,0.099294,-0.582429,0.575306,-1.001335,-0.092479,-0.288591,-0.287202,0.293840,-0.755965,1.658636,1.376260,epoch3
3,0.520527,3.070152,1.107410,0.853044,1.264820,0.490535,1.106828,1.106919,-0.471170,0.656734,0.330802,0.614404,0.819188,1.434374,3.047546,0.803150,epoch4
4,-0.377029,3.329204,1.389947,1.003955,1.065732,1.136646,1.271188,1.073409,-0.470081,0.791998,0.426955,1.255504,0.150832,-0.207679,3.267344,0.815705,epoch5


#### MIN

In [60]:
df_min = df_d[channels].applymap(lambda x: np.min(x)).assign(epochs=epochs)
df_min

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,-62.563165,-44.205089,-39.532387,-45.887008,-42.945967,-30.533564,-24.299576,-66.272047,-37.221867,-31.925681,-39.605170,-44.006126,-53.366752,-112.282318,-31.065010,-32.796245,epoch1
1,-43.525856,-39.202145,-40.628102,-41.505768,-37.756204,-25.621961,-19.241915,-43.453232,-26.904887,-27.560910,-45.350761,-37.008687,-55.679503,-137.627683,-46.818992,-30.600629,epoch2
2,-44.944936,-85.748023,-75.464977,-45.906125,-45.779771,-26.343942,-19.997921,-48.924915,-25.900523,-35.892638,-63.908013,-47.061646,-57.286131,-86.909738,-83.898897,-33.305038,epoch3
3,-67.552821,-105.484506,-103.292212,-111.708651,-102.079751,-49.020262,-50.425961,-118.260457,-53.007384,-50.654543,-78.726479,-97.307314,-108.596331,-138.898798,-102.906165,-82.186696,epoch4
4,-62.318493,-102.679879,-113.981957,-99.602957,-107.591744,-55.588929,-43.712277,-95.698736,-49.045919,-54.256138,-80.161435,-105.362513,-109.179188,-149.776197,-94.971865,-64.434494,epoch5


#### MAX

In [62]:
df_max = df_d[channels].applymap(lambda x: np.min(x)).assign(epochs=epochs)
df_max

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,-62.563165,-44.205089,-39.532387,-45.887008,-42.945967,-30.533564,-24.299576,-66.272047,-37.221867,-31.925681,-39.605170,-44.006126,-53.366752,-112.282318,-31.065010,-32.796245,epoch1
1,-43.525856,-39.202145,-40.628102,-41.505768,-37.756204,-25.621961,-19.241915,-43.453232,-26.904887,-27.560910,-45.350761,-37.008687,-55.679503,-137.627683,-46.818992,-30.600629,epoch2
2,-44.944936,-85.748023,-75.464977,-45.906125,-45.779771,-26.343942,-19.997921,-48.924915,-25.900523,-35.892638,-63.908013,-47.061646,-57.286131,-86.909738,-83.898897,-33.305038,epoch3
3,-67.552821,-105.484506,-103.292212,-111.708651,-102.079751,-49.020262,-50.425961,-118.260457,-53.007384,-50.654543,-78.726479,-97.307314,-108.596331,-138.898798,-102.906165,-82.186696,epoch4
4,-62.318493,-102.679879,-113.981957,-99.602957,-107.591744,-55.588929,-43.712277,-95.698736,-49.045919,-54.256138,-80.161435,-105.362513,-109.179188,-149.776197,-94.971865,-64.434494,epoch5


#### STANDARD DEVIATION

In [63]:
df_stdev = df_d[channels].applymap(lambda x: np.std(x)).assign(epochs=epochs)
df_stdev

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,ch13,ch14,ch15,ch16,epochs
0,19.575639,13.429926,14.128247,14.426470,13.773850,9.338549,8.051992,19.503681,15.624112,10.194209,12.745034,13.594160,17.003731,33.343617,10.482841,11.023413,epoch1
1,24.370998,12.538645,13.813015,15.424608,13.763379,9.565288,7.507496,14.520294,13.186737,9.899756,16.582010,14.304562,18.430571,83.442593,13.733176,11.177925,epoch2
2,19.745394,14.001802,23.108914,16.765496,14.583245,8.847960,7.414642,16.075997,13.260976,11.743435,31.596336,16.553060,32.315650,36.843704,13.840364,12.862808,epoch3
3,22.756774,19.328900,30.290143,33.128423,30.084393,16.297908,10.428283,35.676971,28.015736,18.654146,25.565564,29.768094,32.117508,36.976465,18.801558,25.393117,epoch4
4,23.294195,16.616151,24.558996,25.605826,24.558328,13.691334,9.204229,25.167071,18.419315,14.505816,21.995675,23.937936,39.818854,50.246838,15.879024,19.937958,epoch5
